# My first reconstruction

After FracPty has been installed using the 01-installation notebook everything is ready for a first reconstruction. In this document we will first show you how to perform the reconstruction, and then we will go into all the steps that are being performed.

## Running a reconstruction
Do the following steps to perform a reconstruction
* Open the Spyder program
* Press file/open (or ctrl-o)
* Locate the FracPty folder and select the example_scripts/tutorial_reconstruction.py file
* Press the "Run file" button (see image below) or press F5
![Run button](img/spyder_run_button.png)

Now FracPty will work it's magic and start the first reconstruction. During the reconstruction two windows with the results will be created which we will cover later. For now, take a short brake and wait for the reconstruction to finish.

## Interpreting the output
After the reconstruction has finished your Spyder screen will look similar to this:
![Screen after initial run](img/spyder_after_first_reconstruction.png)
For now it suffices to say that the bottom right part of this window contains the output of the script which shows you how many iterations were performed and at what speed. This is also the place where any errors in the code would appear if anything were to go wrong during execution.

Changes to the code can be made in the left part of the Spyder window. Typically this part of the window is used to set the various parameters of FracPty and to inform it about the location of our data etcetera. Finally, the section on the top right contains other handy tools for more advanced usage. These will not be covered in this tutorial.

As mentioned before, FracPty opens two new figures during the reconstruction. Figure 1 contains the results and should look as follows
![Reconstruction result](img/result_first_reconstruction.png)

It consists of three subfigures. The left one contains the current object estimate. As ptychography is able to recover the complex object, the object has both an intensity and a phase. FracPty represents such data by showing the phase as the color (the the color bar for the scale), and the intensity is encode as the value of each pixes as defined by the HSV color encoding scheme. This means that dark parts of the image represent areas with low intensity and bright parts of the image represent high intensity.

The second subfigure contains the current estimate of the probe beam using the same color coding as for the object estimate. As holds for the object, this estimate will typically improve with each iteration untill it stabilizes at some point.

Finally, the third subfigure shows the error that is estimated at the end of each iteration. This error is computed by the RMS value of the difference between the measured images and the images that would be measured if the object and probe were correct. As can be seen in the example, it takes some 100 iterations for this reconstruction to converge to a solution.

## Analyzing the code
Now that we understand the output of the script, let's take a look at how this output is obtained. In future tutorials we will go into more detail on how to change the behaviour of FracPty such as using a different ptychographic engine or analyzing data from a multi-modal beam.

The tutorial_reconstruction.py script starts with the following

In [1]:
import matplotlib
matplotlib.use('tkagg')
import unittest

from fracPy.ExperimentalData.ExperimentalData import ExperimentalData
from fracPy.Optimizable.Optimizable import Optimizable
from fracPy.engines import ePIE
from fracPy.monitors.Monitor import Monitor

Here the relevant parts of FracPy are loaded, as well as matplotlib for the plotting. In future tutorials we will introduce additional parts that can be loaded. For now it suffices to say that using these classes only we can perform a full reconstruction as you experienced at the start of this tutorial.

The next section of code loads some example data that was bundled with FracPty.

In [ ]:
data = ExperimentalData()
data.loadData('example:ptycho_simulation.hdf5')

 This makes it easy to get started and to troubleshoot any problems with the installation. Later, when you want to try with your own data you can simply replace 'example:ptycho_simulation.hdf5' with the path to your own hdf5 file. This file should be created from your experimental data. This process will be described in a future tutorial.
 
 FracPty can deal with both conventional ptychography and Fourier ptychography and the next line informs FracPty that conventional ptychography should be used in this case.

In [ ]:
data.operationMode = 'CPM'

Next we create the Optimizable object which contains all the parameters that are needed for the reconstruction. Most of these parameters were loaded from the HDF5 file. After this, we create the Monitor. This object allows us to tell FracPty what we would like to see as an output like how frequently the progress window should be updated. For now we will go with the default behaviour.

In [ ]:
opt = Optimizable(data)

monitor = Monitor()

FracPty makes it very easy to switch between reconstruction engines. Exactly how this is done will be covered in a future tutorial. For now, we will take ePIE as an example. It receives the Optimizable, Data and Monitor as inputs such that they can all be tied together.

In [ ]:
engine = ePIE.ePIE(opt, data, monitor)

Finally we set the last two options for the reconstruction, namely the number of iterations to perform and the number of modes the use when solving for the probe.

In [ ]:
engine.numIterations = 200

opt.npsm = 1

Finally, we prepare for the reconstruction which initializes some structures in memory and then we perform the actual reconstruction.

In [ ]:
opt.prepare_reconstruction()
engine.doReconstruction()